In [1]:
from bs4 import BeautifulSoup
import os
import urllib.request
import urllib.parse
import re
import urllib.request, urllib.parse, http.cookiejar
import codecs

In [2]:
#获取网页内容

#读取网页
def getHtml(url):
    cj = http.cookiejar.CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    opener.addheaders = [('User-Agent',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36'),('Cookie', '4564564564564564565646540')]

    urllib.request.install_opener(opener)
    try:
        html_bytes = urllib.request.urlopen(url).read()
    except:
        return '',False
    
    html_string = ''
    try:
        html_string = html_bytes.decode('utf-8')
    except:
        try:
            html_string = html_bytes.decode('gbk')
        except:
            try:
                html_string = html_bytes.decode('gb2312')
            except:
                return '',False
    return html_string,True


In [3]:
#分词
import sys
import jieba
from os import path

re_cdata=re.compile('//<!\[CDATA\[[^>]*//\]\]>',re.I)#处理CDATA
re_doctype=re.compile('<!DOCTYPE HTML PUBLIC[^>]*>',re.I)  
re_doctype2=re.compile('<!DOCTYPE HTML[^>]*>',re.I) 
re_script=re.compile(r'<script[\s\S]+?/script>',re.I)#处理script
re_style=re.compile(r'<style[\s\S]+?/style>',re.I)#处理style
re_div=re.compile('<\s*div[^>]*>[^<]*<\s*/\s*div\s*>',re.I)#处理div
re_br = re.compile('<br\s*?/?>',re.I)  
re_h=re.compile('</?\w+[^>]*>',re.I)#HTML标签
# re_comment=re.compile('<!--[^>]*-->')#HTML注释
re_comment = re.compile('<!--[\s\S]*-->',re.I)  

def processHtml(link, file_content):
    file_content = re_cdata.sub('',file_content)
    file_content = re_doctype.sub('',file_content)
    file_content = re_doctype2.sub('',file_content)
    file_content = re_script.sub('',file_content)
    file_content = re_style.sub('',file_content)
    file_content = re_div.sub('',file_content)
#     file_content = re_br.sub('\n',file_content)
    file_content = re_h.sub('',file_content)
    file_content = re_comment.sub('',file_content)
    file_content = re.sub('[\r\n\t  !©<>“”»《》!:,?。.：？，！、　•"_◆×|()-【】]', '', file_content)
    file_content = re.sub('DOCTYPEdoctypehtml', '', file_content)
#     file_content = re.sub('', '', file_content)
#     print('file_content:',file_content)
    seg_list = jieba.cut_for_search(file_content)
    return link +  ' ' +  '/'.join(seg_list) + '\n'


In [4]:
#爬取层数变换
crawl_turns = 1
#网址列表
rel = 'https://www.baidu.com/'
arr = [[rel]] * 3
success_crwal = True
#爬取网页数量
crawl_total = 500
#保存链接数量
save_total = crawl_total - 1

In [5]:
i = 0
j = 1
arr[j] = []
#遍历列表，爬取所有网页
element = rel
#获取网页内容
html_doc,success_crwal = getHtml(element)
if success_crwal:
    #需爬取网页总数-1
    crawl_total -= 1
    soup = BeautifulSoup(html_doc, 'html.parser')
    #处理后写入文件
    f = codecs.open('./saved_html/' + str(crawl_total) + '.txt',"w",encoding="utf-8")
    f.write(processHtml(element, str(soup)))
    f.close()
    link_arr = [s.extract() for s in soup('a')]
    for element_addr in link_arr:
        try:
            addr = element_addr['href'][0]
            #链接指向网页
            if ((addr == 'h' or addr == '/') and (not element_addr['href'].split()[0].endswith('.apk'))):
                if addr == '/':
                    #相对链接
                    addr = element.split('//')[0] + '//' + element.split('//')[1].split('/')[0] + element_addr['href'].split()[0]
                    #链接不在已访问列表中
                    if addr not in arr[2]:
                        #是符合要求的网页，需要保存的总网页数-1
                        save_total -= 1
                        arr[j].append(addr)
                        arr[2].append(addr)
                        pass
                    pass
                else:
                    #绝对链接
                    addr = element_addr['href'].split()[0]
                    #链接不在已访问列表中
                    if addr not in arr[2]:
                        #是符合要求的网页，需要保存的总网页数-1
                        save_total -= 1
                        arr[j].append(addr)
                        arr[2].append(addr)
                        pass
                    pass
        except:
            pass
        pass
    pass
pass


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\xlc\AppData\Local\Temp\jieba.cache
Loading model cost 0.815 seconds.
Prefix dict has been built succesfully.


In [6]:
while crawl_total > 0:
    i = crawl_turns
    j = (i + 1) % 2
    arr[j] = []
    crawl_turns = (crawl_turns + 1) % 2
    #遍历列表，爬取所有网页
    for element in arr[i]:
        #获取网页内容
        html_doc,success_crwal = getHtml(element)
        if success_crwal:
            #需爬取网页总数-1
            if crawl_total <= 0:
                break
            crawl_total -= 1
            print('crawl_total:',crawl_total)
            soup = BeautifulSoup(html_doc, 'html.parser')
            #处理后写入文件
            f = codecs.open('./saved_html/' + str(crawl_total) + '.txt',"w",encoding="utf-8")
            f.write(processHtml(element, str(soup)))
            f.close()
            if save_total <= 0:
                #需要保存的网页总数足够，不用继续保存
                continue
            link_arr = [s.extract() for s in soup('a')]
            for element_addr in link_arr:
                try:
                    addr = element_addr['href'][0]
                    #链接指向网页
                    if ((addr == 'h' or addr == '/') and (not element_addr['href'].split()[0].endswith('.apk'))):
                        if addr == '/':
                            #相对链接
                            addr = element.split('//')[0] + '//' + element.split('//')[1].split('/')[0] + element_addr['href'].split()[0]
                            #链接不在已访问列表中
                            if addr not in arr[2]:
                                #是符合要求的网页，需要保存的总网页数-1
                                save_total -= 1
                                arr[j].append(addr)
                                arr[2].append(addr)
                                pass
                            pass
                        else:
                            #绝对链接
                            addr = element_addr['href'].split()[0]
                            #链接不在已访问列表中
                            if addr not in arr[2]:
                                #是符合要求的网页，需要保存的总网页数-1
                                save_total -= 1
                                arr[j].append(addr)
                                arr[2].append(addr)
                                pass
                            pass
                except:
                    pass
                pass
            pass
        else:
            save_total += 1
            pass
        pass


crawl_total: 498
crawl_total: 497
crawl_total: 496
crawl_total: 495
crawl_total: 494
crawl_total: 493
crawl_total: 492
crawl_total: 491
crawl_total: 490
crawl_total: 489
crawl_total: 488
crawl_total: 487
crawl_total: 486
crawl_total: 485
crawl_total: 484
crawl_total: 483
crawl_total: 482
crawl_total: 481
crawl_total: 480
crawl_total: 479
crawl_total: 478
crawl_total: 477
crawl_total: 476
crawl_total: 475
crawl_total: 474
crawl_total: 473
crawl_total: 472
crawl_total: 471
crawl_total: 470
crawl_total: 469
crawl_total: 468
crawl_total: 467
crawl_total: 466
crawl_total: 465
crawl_total: 464
crawl_total: 463
crawl_total: 462
crawl_total: 461
crawl_total: 460
crawl_total: 459
crawl_total: 458
crawl_total: 457
crawl_total: 456
crawl_total: 455
crawl_total: 454
crawl_total: 453
crawl_total: 452
crawl_total: 451
crawl_total: 450
crawl_total: 449
crawl_total: 448
crawl_total: 447
crawl_total: 446
crawl_total: 445
crawl_total: 444
crawl_total: 443
crawl_total: 442
crawl_total: 441
crawl_total: 4

crawl_total: 10
crawl_total: 9
crawl_total: 8
crawl_total: 7
crawl_total: 6
crawl_total: 5
crawl_total: 4
crawl_total: 3
crawl_total: 2
crawl_total: 1
crawl_total: 0
